In [4]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler

# Load and preprocess data
df = pd.read_csv('stock_post_eda.csv')
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')

# Calculate daily returns
df['Daily_Return'] = df['Closing_Price'].pct_change()

# Create target variable (1 if price went up, 0 if it went down or stayed the same)
df['Target'] = (df['Daily_Return'] > 0).astype(int)

# Create features (you can add more based on your analysis)
feature_columns = ['Opening_Price', 'High_Price', 'Low_Price', 'Volume_Millions']

In [5]:
# Normalize features
scaler = StandardScaler()
df[feature_columns] = scaler.fit_transform(df[feature_columns])

In [6]:
# Create sequences
def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:(i + sequence_length)])
        y.append(data.Target.iloc[i + sequence_length])
    return np.array(X), np.array(y)

sequence_length = 10
X, y = create_sequences(df[feature_columns + ['Target']], sequence_length)

In [7]:
# Split data
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [8]:
# Build model
model = Sequential([
    LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    Dropout(0.2),
    LSTM(50, activation='relu', return_sequences=False),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
# Train model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/100
184/184 [==============================] - 7s 14ms/step - loss: 0.6799 - accuracy: 0.5732 - val_loss: 0.6900 - val_accuracy: 0.5374
Epoch 2/100
184/184 [==============================] - 2s 10ms/step - loss: 0.6776 - accuracy: 0.5766 - val_loss: 0.6907 - val_accuracy: 0.5374
Epoch 3/100
184/184 [==============================] - 2s 10ms/step - loss: 0.6769 - accuracy: 0.5776 - val_loss: 0.6900 - val_accuracy: 0.5374
Epoch 4/100
184/184 [==============================] - 2s 10ms/step - loss: 0.6764 - accuracy: 0.5776 - val_loss: 0.6899 - val_accuracy: 0.5374
Epoch 5/100
184/184 [==============================] - 2s 10ms/step - loss: 0.6769 - accuracy: 0.5769 - val_loss: 0.6900 - val_accuracy: 0.5374
Epoch 6/100
184/184 [==============================] - 2s 10ms/step - loss: 0.6759 - accuracy: 0.5769 - val_loss: 0.6901 - val_accuracy: 0.5374
Epoch 7/100
184/184 [==============================] - 2s 10ms/step - loss: 0.6759 - accuracy: 0.5778 - val_loss: 0.6901 - val_accuracy:

In [10]:
# Evaluate model
y_pred = (model.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

58/58 [==============================] - 1s 4ms/step
Accuracy: 0.5068
F1 Score: 0.4309
